# Análisis Exploratorio de Datos (EDA) - Detector de División de Transacciones

## Objetivo
Familiarizarse con los datos de transacciones y entender su estructura:
- Shape, tipos de datos, valores nulos, valores atípicos
- Distribuciones de montos y tipos de transacciones
- Identificación de patrones iniciales

## Dataset
- **Archivos**: `sample_data_0006_part_00.parquet` y `sample_data_0007_part_00.parquet`
- **Ubicación**: `../data/`

In [1]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pathlib import Path

# Configuración para visualizaciones
plt.style.use('default')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

# Configuración de pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

print("✅ Librerías importadas correctamente")

✅ Librerías importadas correctamente


## 1. Carga de Datos

Vamos a cargar los dos archivos parquet y verificar si tienen la misma estructura para poder unirlos.

In [3]:
# Definir rutas de los archivos
data_path = Path("../data/")
file1 = data_path / "sample_data_0006_part_00.parquet"
file2 = data_path / "sample_data_0007_part_00.parquet"

# Cargar cada archivo por separado para explorar
print("📁 Cargando archivos...")
df1 = pd.read_parquet(file1)
df2 = pd.read_parquet(file2)

print(f"✅ Archivo 1 cargado: {file1.name}")
print(f"   Shape: {df1.shape}")
print(f"✅ Archivo 2 cargado: {file2.name}")
print(f"   Shape: {df2.shape}")

📁 Cargando archivos...
✅ Archivo 1 cargado: sample_data_0006_part_00.parquet
   Shape: (10758418, 8)
✅ Archivo 2 cargado: sample_data_0007_part_00.parquet
   Shape: (10758500, 8)


In [6]:
# Verificar estructura de ambos archivos
print("🔍 Verificando estructura de los archivos...\n")

print("📋 COLUMNAS ARCHIVO 1:")
print(f"Columnas: {list(df1.columns)}\n")

print("📋 COLUMNAS ARCHIVO 2:")
print(f"Columnas: {list(df2.columns)}\n")

# Verificar si las columnas son iguales
columnas_iguales = list(df1.columns) == list(df2.columns)
print(f"¿Las columnas son iguales? {columnas_iguales}")

if columnas_iguales:
    print("✅ Los archivos tienen la misma estructura, se pueden unir")
else:
    print("⚠️ Los archivos tienen diferente estructura")

🔍 Verificando estructura de los archivos...

📋 COLUMNAS ARCHIVO 1:
Columnas: ['merchant_id', '_id', 'subsidiary', 'transaction_date', 'account_number', 'user_id', 'transaction_amount', 'transaction_type']

📋 COLUMNAS ARCHIVO 2:
Columnas: ['merchant_id', '_id', 'subsidiary', 'transaction_date', 'account_number', 'user_id', 'transaction_amount', 'transaction_type']

¿Las columnas son iguales? True
✅ Los archivos tienen la misma estructura, se pueden unir


In [ ]:
# Unir los datasets
if columnas_iguales:
    print("🔗 Uniendo los datasets...")
    df = pd.concat([df1, df2], ignore_index=True)
    print(f"✅ Dataset unificado creado")
    print(f"   Shape final: {df.shape}")
    print(f"   Registros archivo 1: {len(df1)}")
    print(f"   Registros archivo 2: {len(df2)}")
    print(f"   Total registros: {len(df)}")
    
    # Liberar memoria de los dataframes individuales
    del df1, df2
else:
    print("❌ No se pueden unir los archivos debido a diferencias en estructura")

🔗 Uniendo los datasets...
✅ Dataset unificado creado
   Shape final: (21516918, 8)
   Registros archivo 1: 10758418
   Registros archivo 2: 10758500
   Total registros: 21516918
